In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /Users/Madison/opt/anaconda3/lib/python3.8/site-packages (0.0)


In [2]:
import pandas as pd
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [3]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()

# Select your features (columns)

In [4]:
# Set features. This will also be used as your x values.
# Upon review of the exoplanet archive website the following variables appear to have impact
# on if a exoplanet is confirmed. Error were not taken into account as in most cases the 
# error was negligable compared to the value. koi_time0bk was removed as it is a time it 
# was observed not a feature of the object. 
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_impact', 'koi_duration',
        'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol','koi_model_snr', 
        'koi_tce_plnt_num', 'koi_steff', 'koi_slogg',
       'koi_srad', 'ra', 'dec','koi_kepmag']]
selected_features.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,0.586,4.50700,874.8,2.83,443,9.11,25.8,2,5455,4.467,0.927,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,0.969,1.78220,10829.0,14.60,638,39.30,76.3,1,5853,4.544,0.868,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,1.276,2.40641,8079.2,33.46,1395,891.96,505.6,1,5805,4.564,0.791,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,0.701,1.65450,603.3,2.75,1406,926.16,40.9,1,6031,4.438,1.046,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,0.762,3.14020,686.0,2.77,1160,427.65,40.2,2,6046,4.486,0.972,296.28613,48.224670,15.714


# Create a Train Test Split

Use `koi_disposition` for the y values

In [5]:
y = df["koi_disposition"]
y_binary_encoded = pd.get_dummies(y)

In [6]:
from sklearn.model_selection import train_test_split

#spliting training data, setting back to x and y for ease of modeling
X_train, X_test, y_train, y_test = train_test_split(selected_features, y_binary_encoded, random_state=57)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [7]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
# only X variable needs to be scaled since y is all binary encoding
# using training data to create the scaler 
X_MinMaxScaler = MinMaxScaler().fit(X_train)

In [8]:
X_train_scaled = X_MinMaxScaler.transform(X_train)
X_test_scaled = X_MinMaxScaler.transform(X_test)

# Train the Model



In [10]:
# first model tested is linear regression to get a base comparison 
from sklearn.linear_model import LinearRegression
model1 = LinearRegression()
model1.fit(X_train_scaled, y_train)

LinearRegression()

In [11]:
print(f"Training Data Score: {model1.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model1.score(X_test_scaled, y_test)}")

Training Data Score: 0.4665212178492852
Testing Data Score: 0.4541913012928085


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [12]:
# Create the GridSearchCV model
# using linear regression grid search params
from sklearn.model_selection import GridSearchCV
param_grid1 = {'fit_intercept': [True, False],
              'normalize': [True, False]}
grid1 = GridSearchCV(model1, param_grid1, verbose=3)

In [14]:
# Train the model with GridSearch
grid1.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] fit_intercept=True, normalize=True ..............................
[CV] .. fit_intercept=True, normalize=True, score=0.456, total=   0.0s
[CV] fit_intercept=True, normalize=True ..............................
[CV] .. fit_intercept=True, normalize=True, score=0.463, total=   0.0s
[CV] fit_intercept=True, normalize=True ..............................
[CV] .. fit_intercept=True, normalize=True, score=0.465, total=   0.0s
[CV] fit_intercept=True, normalize=True ..............................
[CV] .. fit_intercept=True, normalize=True, score=0.464, total=   0.0s
[CV] fit_intercept=True, normalize=True ..............................
[CV] .. fit_intercept=True, normalize=True, score=0.465, total=   0.0s
[CV] fit_intercept=True, normalize=False .............................
[CV] . fit_intercept=True, normalize=False, score=0.456, total=   0.0s
[CV] fit_intercept=True, normalize=False .............................
[CV] . fit_interc

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished


GridSearchCV(estimator=LinearRegression(),
             param_grid={'fit_intercept': [True, False],
                         'normalize': [True, False]},
             verbose=3)

In [15]:
print(grid1.best_params_)
print(grid1.best_score_)

{'fit_intercept': True, 'normalize': True}
0.4624463119307626


# Save the Model

In [16]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'mbfisher_model1.sav'
joblib.dump(grid1, filename)

['mbfisher_model1.sav']

In [17]:
predict_values = grid1.predict(X_test_scaled)
predict_values

array([[ 0.4042637 ,  0.50907951,  0.0866568 ],
       [ 0.52302536,  0.31659281,  0.16038183],
       [ 0.46444305,  0.41577359,  0.11978337],
       ...,
       [ 0.14970787,  0.15939292,  0.69089921],
       [-0.15457324, -0.19289097,  1.34746421],
       [ 0.18399529,  0.28728297,  0.52872173]])